In [1]:
import os
import json
import pandas as pd



/tmp/ipykernel_7167/2053734199.py:3: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
df_answers = pd.read_csv('reddit_answers_big.csv', sep = ';')

In [3]:
df_top_votes= df_answers.groupby('q_id')['votes'].idxmax()

In [4]:
df_top_answers = df_answers.loc[df_top_votes]

In [5]:
df_top_answers.rename(columns = {'text':'answers'}, inplace = True)
df_top_answers.rename(columns = {'q_id':'id'}, inplace = True)
df_top_answers.rename(columns = {'votes':'answer_votes'}, inplace = True)

In [6]:
df_questions = pd.read_csv('reddit_questions.csv', sep = ';')
df_questions.rename(columns = {'votes':'question_votes'}, inplace = True)
df_questions.rename(columns = {'text':'question'}, inplace = True)

In [7]:
df_answers.rename(columns = {'text':'question'}, inplace = True)
df_answers.rename(columns = {'votes':'question_votes'}, inplace = True) 

In [8]:
merged_df = df_questions.merge(df_top_answers, on = 'id')

In [9]:
merged_df.drop(columns = ['timestamp', 'datetime', 'Unnamed: 0'], inplace = True)

In [10]:
merged_df= merged_df.reindex(columns = ['id', 'question',  'answers', 'question_votes', 'answer_votes'])

In [11]:
merged_df = merged_df.sort_values(by = 'answer_votes', ascending = False)

In [12]:
merged_df  = merged_df [:1000]

In [13]:
merged_df = merged_df[:500]

In [14]:
merged_df

,id,question,answers,question_votes,answer_votes
75274,fkzaca,What is something that has aged well?,The word cool,66093,99398.0
167081,a0a4cd,What's the most amazing thing about the universe?,"It must be true that either It didn't exist, ...",81862,86042.0
140939,d0jjc2,The 2010's decade will be over in 4 months. Wh...,The social media explosion,113254,85936.0
128868,aqf3bi,"You are offered $1,000,000 USD if you can hide...","Easy, ask the CIA to hold them...those two don...",81908,85693.0
18363,bvdaci,What's classy if you're rich but trashy if you...,The most expensive thing you own is a really o...,66102,85568.0
...,...,...,...,...,...
139296,9w884k,"What's your """"thank god that's over with and I...","Finding a first job. It was at least 90% luck,...",33470,44574.0
32384,82630j,"What profession was once highly respected, but...",Bank teller used to be a foot in the door to t...,46027,44549.0
141294,a2zwpp,"What's a rule that was implemented somewhere, ...",My company has a strict no alcohol policy. You...,52721,44504.0
155069,cjr8nm,What folklore creature do you think really exi...,The Kraken is almost definitely an extra thicc...,51781,44499.0


In [15]:
questions, answers = merged_df['question'], merged_df['answers']

In [16]:
qa_openai_format = [{"messages": [{'role': 'system', 'content': 'Marv is a factual chatbot and reddit expert who likes to answer with bullets'},
                                  {'role': 'user', 'content': q},
                                  {'role': 'assistant', 'content': a}]}
                    for q, a in zip(questions, answers)]


In [17]:
qa_openai_format[3]

{'messages': [{'role': 'system',
   'content': 'Marv is a factual chatbot and reddit expert who likes to answer with bullets'},
  {'role': 'user',
   'content': 'You are offered $1,000,000 USD if you can hide a pair of car keys from the entirety of the FBI force for 7 days. Where do you hide the keys?'},
  {'role': 'assistant',
   'content': "Easy, ask the CIA to hold them...those two don't share shit."}]}

In [18]:
with open('training_data.jsonl', 'w') as f:
    for entry in qa_openai_format:
        f.write(json.dumps(entry))
        f.write('\n')

In [19]:
# Format error checks

from collections import defaultdict


format_errors = defaultdict(int)

data_path = "training_data.jsonl"

with open(data_path, "r") as f:
    dataset= [json.loads(line) for line in f]
    
print(f"Loaded {len(dataset)} examples")

for ex in dataset:
    if not isinstance(ex, dict):
        format_errors["data_type"] += 1
        continue
        
    messages = ex.get("messages", None)
    if not messages:
        format_errors["missing_messages_list"] += 1
        continue
        
    for message in messages:
        if "role" not in message or "content" not in message:
            format_errors["message_missing_key"] += 1
        
        if any(k not in ("role", "content", "name", "function_call", "weight") for k in message):
            format_errors["message_unrecognized_key"] += 1
        
        if message.get("role", None) not in ("system", "user", "assistant", "function"):
            format_errors["unrecognized_role"] += 1
            
        content = message.get("content", None)
        function_call = message.get("function_call", None)
        
        if (not content and not function_call) or not isinstance(content, str):
            format_errors["missing_content"] += 1
    
    if not any(message.get("role", None) == "assistant" for message in messages):
        format_errors["example_missing_assistant_message"] += 1

if format_errors:
    print("Found errors:")
    for k, v in format_errors.items():
        print(f"{k}: {v}")
else:
    print("No errors found")

Loaded 500 examples
No errors found


In [29]:
import openai
from openai import OpenAI

client = OpenAI(api_key = "")

In [31]:
client.files.create(file=open("training_data.jsonl", "rb"), purpose="fine-tune")

FileObject(id='file-pqN0u7lBPpI82Bv4udlgpejD', bytes=235808, created_at=1713506896, filename='training_data.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

In [ ]:
file-pqN0u7lBPpI82Bv4udlgpejD

In [33]:
client.fine_tuning.jobs.create(training_file="file-pqN0u7lBPpI82Bv4udlgpejD", model="gpt-3.5-turbo")

FineTuningJob(id='ftjob-i3343i8eFZ1lbBYoWKDJvL4w', created_at=1713507047, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-To0uUA7DnRus6HYSgl3vvG8G', result_files=[], seed=1240979456, status='validating_files', trained_tokens=None, training_file='file-pqN0u7lBPpI82Bv4udlgpejD', validation_file=None, integrations=[], user_provided_suffix=None)

In [34]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-i3343i8eFZ1lbBYoWKDJvL4w', created_at=1713507047, error=Error(code=None, message=None, param=None, error=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs=3, batch_size=1, learning_rate_multiplier=2), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-To0uUA7DnRus6HYSgl3vvG8G', result_files=[], seed=1240979456, status='running', trained_tokens=None, training_file='file-pqN0u7lBPpI82Bv4udlgpejD', validation_file=None, integrations=[], user_provided_suffix=None)], object='list', has_more=False)

In [39]:
job_id= 'ftjob-i3343i8eFZ1lbBYoWKDJvL4w'

In [37]:
import signal
import datetime


def signal_handler(sig, frame):
    status = client.fine_tuning.jobs.retrieve(job_id).status
    print(f"Stream interrupted. Job is still {status}.")
    return


signal.signal(signal.SIGINT, signal_handler)

events = client.fine_tuning.jobs.list_events(fine_tuning_job_id='ftjob-i3343i8eFZ1lbBYoWKDJvL4w')
try:
    for event in events:
        print(
            f'{datetime.datetime.fromtimestamp(event.created_at)} {event.message}'
        )
except Exception:
    print("Stream interrupted (client disconnected).")

2024-04-19 14:27:07 Step 564/1500: training loss=2.20
2024-04-19 14:27:05 Step 563/1500: training loss=1.92
2024-04-19 14:27:05 Step 562/1500: training loss=2.24
2024-04-19 14:27:03 Step 561/1500: training loss=0.91
2024-04-19 14:27:01 Step 560/1500: training loss=1.73
2024-04-19 14:27:01 Step 559/1500: training loss=1.36
2024-04-19 14:26:59 Step 558/1500: training loss=2.58
2024-04-19 14:26:57 Step 557/1500: training loss=1.28
2024-04-19 14:26:55 Step 556/1500: training loss=1.05
2024-04-19 14:26:55 Step 555/1500: training loss=1.44
2024-04-19 14:26:53 Step 554/1500: training loss=4.25
2024-04-19 14:26:51 Step 553/1500: training loss=2.61
2024-04-19 14:26:49 Step 552/1500: training loss=1.54
2024-04-19 14:26:49 Step 551/1500: training loss=1.97
2024-04-19 14:26:47 Step 550/1500: training loss=2.37
2024-04-19 14:26:45 Step 549/1500: training loss=1.71
2024-04-19 14:26:45 Step 548/1500: training loss=1.86
2024-04-19 14:26:43 Step 547/1500: training loss=1.81
2024-04-19 14:26:41 Step 546

In [40]:
import time

status = client.fine_tuning.jobs.retrieve(job_id).status
if status not in ["succeeded", "failed"]:
    print(f"Job not in terminal status: {status}. Waiting.")
    while status not in ["succeeded", "failed"]:
        time.sleep(2)
        status = client.fine_tuning.jobs.retrieve(job_id).status
        print(f"Status: {status}")
else:
    print(f"Finetune job {job_id} finished with status: {status}")
print("Checking other finetune jobs in the subscription.")
result = client.fine_tuning.jobs.list()
print(f"Found {len(result.data)} finetune jobs.")

Job not in terminal status: running. Waiting.
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: running
Status: ru

In [41]:
fine_tuned_model = result.data[0].fine_tuned_model
print(fine_tuned_model)


ft:gpt-3.5-turbo-0125:personal::9FcMkILC


In [42]:
system_prompt = "Marv is a factual chatbot and reddit expert who likes to answer with bullets"
user_question = "Give me the dumbest thing you've ever done."

In [43]:
response = client.chat.completions.create(
    model = "ft:gpt-3.5-turbo-0125:personal::9FcMkILC",
    messages = [
        {'role':'system', "content": system_prompt},
        {'role':'user', "content": user_question}
    ]
)

print(response.choices[0].message.content)

'Tried to microwave and egg to see if I could make a hard boiled egg without boiling it. It exploded. And the left front burner. '
